[View in Colaboratory](https://colab.research.google.com/github/thayssa1186/MnistKeras/blob/master/mnistcnnkeras.ipynb)

In [2]:
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import model_from_json

Using TensorFlow backend.


In [0]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')
X_train = X_train / 255
X_test = X_test / 255
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [0]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(1, 28, 28), activation='relu', data_format='channels_first'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [0]:
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# check 5 epochs
early_stop = EarlyStopping(monitor='val_loss', patience=5, mode='min') 

callbacks_list = [checkpoint, early_stop]

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test),  epochs=150, batch_size=128,  verbose=2, callbacks=callbacks_list)

scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Train on 60000 samples, validate on 10000 samples
Epoch 1/150
 - 163s - loss: 0.2134 - acc: 0.9372 - val_loss: 0.0713 - val_acc: 0.9785

Epoch 00001: val_loss improved from inf to 0.07131, saving model to weights.best.hdf5
Epoch 2/150
 - 162s - loss: 0.0706 - acc: 0.9780 - val_loss: 0.0490 - val_acc: 0.9840

Epoch 00002: val_loss improved from 0.07131 to 0.04902, saving model to weights.best.hdf5
Epoch 3/150
 - 162s - loss: 0.0505 - acc: 0.9846 - val_loss: 0.0418 - val_acc: 0.9861

Epoch 00003: val_loss improved from 0.04902 to 0.04184, saving model to weights.best.hdf5
Epoch 4/150
 - 162s - loss: 0.0404 - acc: 0.9871 - val_loss: 0.0353 - val_acc: 0.9879

Epoch 00004: val_loss improved from 0.04184 to 0.03526, saving model to weights.best.hdf5
Epoch 5/150
 - 162s - loss: 0.0330 - acc: 0.9894 - val_loss: 0.0323 - val_acc: 0.9900

Epoch 00005: val_loss improved from 0.03526 to 0.03232, saving model to weights.best.hdf5
Epoch 6/150
 - 162s - loss: 0.0257 - acc: 0.9920 - val_loss: 0.0362 -

In [9]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [10]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [11]:
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# calculate predictions
#predictions = loaded_model.predict(X_test)
#y_classes = predictions.argmax(axis=-1)

preds = loaded_model.predict_classes(X_test)
prob = loaded_model.predict_proba(X_test)

# evaluate the model
scores = loaded_model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (loaded_model.metrics_names[1], scores[1]*100))

10000/10000 [==============================] - 17s 2ms/step

acc: 99.79%
